In [37]:
import requests
from bs4 import BeautifulSoup
import gzip
import os

def scrape_and_extract(url):
    # Scrape the source code of the website page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all .txt.gz files on the page
    txt_gz_links = soup.find_all('a', href=lambda x: x.endswith('.txt.gz'))

    os.makedirs("files")

    # Download and extract each .txt.gz file
    for link in txt_gz_links:
        download_url = link['href']
        filename = os.path.join("files", download_url.split('/')[-1])

        # Download the .txt.gz file
        response = requests.get(url + download_url)
        with open(filename, 'wb') as f:
            f.write(response.content)

        # Extract the contents of the .txt.gz file
        with gzip.open(filename, 'rb') as f:
            try:
                extracted_content = f.read().decode('utf-8')
            except UnicodeDecodeError:
                # UTF-8 decoding failed, try ISO-8859-1
                extracted_content = f.read().decode('ISO-8859-1')

            try:
                with open(filename.replace('.txt.gz', '.txt'), 'w', encoding='utf-8') as f:
                    f.write(extracted_content)
            except UnicodeEncodeError:
                # UTF-8 encoding failed, write as-is without encoding
                with open(filename.replace('.txt.gz', '.txt'), 'w') as f:
                    f.write(extracted_content)       

        # Delete the original .txt.gz file
        os.remove(filename)

In [38]:
url = 'https://mailman.videolan.org/pipermail/vlc-devel/'
scrape_and_extract(url)